In [1]:
!java -version

openjdk version "1.8.0_362-362"
OpenJDK Runtime Environment (build 1.8.0_362-362-b09)
OpenJDK 64-Bit Server VM (build 25.362-b09, mixed mode)


# Java instalado en el cluster
**openjdk version "1.8.0_362"**

**OpenJDK Runtime Environment (build 1.8.0_362-8u372-ga~us1-0ubuntu1~22.04-b09)**

**OpenJDK 64-Bit Server VM (build 25.362-b09, mixed mode)**

In [9]:
!python --version

Python 3.8.13
23/09/14 10:36:43 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
23/09/14 10:36:43 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:218)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:923)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:154)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:262)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:169)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.net

# Python
Python 3.8.13

In [2]:
!pip install pyspark==3.3.1

# pyspark
**Requirement already satisfied: pyspark==3.3.1 in /opt/conda/lib/python3.8/site-packages (3.3.1)**

**Requirement already satisfied: py4j==0.10.9.5 in /opt/conda/lib/python3.8/site-packages (from pyspark==3.3.1) (0.10.9.5)**

In [3]:
from pyspark.sql import SparkSession
import os
import subprocess

dst_lh_url  = os.environ.get('DST_LH_URL', 'spark://spark-master-0.spark-headless.demml.svc.cluster.local:7077')
dst_lh_appn = os.environ.get('DST_LH_APPN', 'iris_naive_bayes_classification_pyspark_EDU_6')
my_pod_ip = subprocess.run(['hostname', '-I'], stdout=subprocess.PIPE).stdout.decode('utf-8').strip(' \n\t')

spark = SparkSession.builder \
    .master(dst_lh_url) \
    .appName(dst_lh_appn) \
    .config('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.1.0,org.mariadb.jdbc:mariadb-java-client:2.3.0,org.postgresql:postgresql:42.5.4,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257,software.amazon.awssdk:s3:2.17.257,software.amazon.awssdk:iam:2.17.257,org.apache.hadoop:hadoop-aws:3.2.3') \
    .config('spark.sql.extensions','org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
    .config('spark.sql.catalog.spark_catalog','org.apache.iceberg.spark.SparkSessionCatalog') \
    .config('spark.sql.catalog.spark_catalog.catalog-impl','org.apache.iceberg.jdbc.JdbcCatalog') \
    .config('spark.sql.catalog.spark_catalog.uri','jdbc:postgresql://postgresql:5432/iceberg') \
    .config('spark.driver.host', my_pod_ip) \
    .config('spark.sql.catalog.spark_catalog.jdbc.useSSL','false') \
    .config('spark.sql.catalog.spark_catalog.jdbc.user','iceberg') \
    .config('spark.sql.catalog.spark_catalog.jdbc.password','iceberg') \
    .config('spark.sql.catalog.spark_catalog.warehouse','s3a://warehouse/') \
    .config('spark.sql.catalog.spark_catalog.s3.endpoint','http://minio:9000') \
    .config('spark.sql.catalog.spark_catalog.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO') \
    .config('spark.sql.defaultCatalog','spark_catalog') \
    .config('spark.hadoop.fs.s3a.endpoint','http://minio:9000') \
    .config('spark.hadoop.fs.s3a.access.key','admin') \
    .config('spark.hadoop.fs.s3a.secret.key','t4bl4red0nd4') \
    .config('spark.hadoop.fs.s3a.path.style.access','true') \
    .config('spark.hadoop.fs.s3a.impl','org.apache.hadoop.fs.s3a.S3AFileSystem') \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .getOrCreate()
spark

23/09/14 10:23:06 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


<pyspark.sql.session.SparkSession object at 0x7f1e5bfa1be0>

In [4]:
!ls -all

total 256
drwxrwsr-x. 3 coder coder   4096 Sep 14 10:21 .
drwxrwsr-x. 8 coder coder   4096 Sep 13 09:41 ..
-rw-rw-r--. 1 coder coder      0 Sep 13 07:01 .gitkeep
drwxrwsr-x. 2 coder coder   4096 Sep 13 09:41 .ipynb_checkpoints
-rw-rw-r--. 1 coder coder  21440 Sep 13 12:10 iris-naive-bayes-classification-with-pyspark.ipynb
-rw-r--r--. 1 coder coder   3858 Sep 14 10:18 iris.csv
-rw-rw-r--. 1 coder coder 218836 Sep 14 10:21 test_conexion_spark.ipynb


In [5]:
df = spark.read.csv("iris.csv", header=True, inferSchema=True)

23/09/14 10:23:13 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (172.21.174.108 executor 1): java.io.FileNotFoundException: 
File file:/home/coder/eromero/iris-pyspark/notebooks/iris.csv does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:661)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:212)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:270)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sq

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 df = spark.read.csv("iris.csv", header=True, inferSchema=True)                               │
│   2                                                                                              │
│                                                                                                  │
│ /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/pyspark/sql/readwriter.py:535 in   │
│ csv                                                                                              │
│                                                                                                  │
│    532 │   │   │   path = [path]                                                                 │
│    533 │   │   if type(path) == list:                                                            │
│    534 │   │   │   assert self._spark._sc._jvm is not None                                       │
│ ❱  535 │   │   │   return self._df(self._jreader.csv(self._spark._sc._jvm.PythonUtils.toSeq(pat  │
│    536 │   │   elif isinstance(path, RDD):                                                       │
│    537 │   │   │                                                                                 │
│    538 │   │   │   def func(iterator):                                                           │
│                                                                                                  │
│ /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/py4j/java_gateway.py:1321 in       │
│ __call__                                                                                         │
│                                                                                                  │
│   1318 │   │   │   proto.END_COMMAND_PART                                                        │
│   1319 │   │                                                                                     │
│   1320 │   │   answer = self.gateway_client.send_command(command)                                │
│ ❱ 1321 │   │   return_value = get_return_value(                                                  │
│   1322 │   │   │   answer, self.gateway_client, self.target_id, self.name)                       │
│   1323 │   │                                                                                     │
│   1324 │   │   for temp_arg in temp_args:                                                        │
│                                                                                                  │
│ /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/pyspark/sql/utils.py:190 in deco   │
│                                                                                                  │
│   187 def capture_sql_exception(f: Callable[..., Any]) -> Callable[..., Any]:                    │
│   188 │   def deco(*a: Any, **kw: Any) -> Any:                                                   │
│   189 │   │   try:                                                                               │
│ ❱ 190 │   │   │   return f(*a, **kw)                                                             │
│   191 │   │   except Py4JJavaError as e:                                                         │
│   192 │   │   │   converted = convert_exception(e.java_exception)                                │
│   193 │   │   │   if not isinstance(converted, UnknownException):                                │
│                                                                                                  │
│ /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/py4j/protocol.py:326 in            │
│ get_return_value                                           

In [ ]:
spark.stop()